# My College Basketball Data

In [1]:
#import
import csv
import numpy as np
import torch 

In [2]:
#set up dictionary for team number and team name
teamidentities = {}
with open('Teams.csv') as csvfile:
    readCSV = csv.reader(csvfile, delimiter=',')
    
    for row in readCSV:
        teamidentities.update( {row[0]: row[1]})
    
    #deletes first key 
    del teamidentities["Team_Id"]
    
    #print(teamidentities)

In [3]:
#read and data split tourney seeds
#1985-2017

trainingDataSetSeeds = []
testDataSetSeeds = []
validationDataSetSeeds = []

with open('TourneySeeds.csv') as csvfile:
    readCSV = csv.reader(csvfile, delimiter=',')
   
    for row in readCSV:
        if row[0] == "2016":
            testDataSetSeeds.append(row)
        elif row[0] == "2017":
            validationDataSetSeeds.append(row)
        else:
            trainingDataSetSeeds.append(row)
            
#     print(trainingDataSetSeeds)
#     print(testDataSetSeeds)
#     print(validationDataSetSeeds)

In [4]:
#read and data split regular season stats
#2003-2015

trainingDataSet = []
testDataSet = []
validationDataSet = []

def dataSplit():
    for row in readCSV:
        if row[0] == "2014":
            testDataSet.append(row)
        elif row[0] == "2015":
            validationDataSet.append(row)
        else:
            trainingDataSet.append(row)
            
with open('RegularSeasonDetailedResults.csv') as csvfile:
    readCSV = csv.reader(csvfile, delimiter=',')
    
    dataSplit()

In [5]:
#read and data split tournament stats
#2003-2016

remainingDataSet = []
with open('TourneyDetailedResults.csv') as csvfile:
    readCSV = csv.reader(csvfile, delimiter=',')
    
    dataSplit()

#used dataSplit() to assemble trainingDataSet, testDataSet, validationDataSet by combining data from RegularSeasonedDetailedResults.csv and TourneyDetailedResults
#trainingDataSet made up of data from RSDR years of 2003-2013 and TDR years 2003-2013, 2016
#testDataSet made up of data from RSDR years 2014 and TDR years 2014
#validationDataSet made up of data from RSDR years 2015 and TDR years 2015

del trainingDataSet[0]
del trainingDataSet[65920]
# print(trainingDataSet)
# print(testDataSet)
# print(validationDataSet)


In [6]:
#extracting features for each DataSet
#first half of the feature vector is winning team stats and last half is losing team stats
#features taken out: [Wteam, Wfgm, Wfga, Wfgm3, Wfga3, Wftm, Wfta, Wor, Wdr, Wast, Wto, Wstl, Wblk, Wpf, Lteam, Lfgm, Lfga, Lfgm3, Lfga3, Lftm, Lfta, Lor, Ldr, Last, Lto, Lstl, Lblk, Lpf]
#score is taken out

#function to featureExtract
def featureExtract(dataSetArray):
    featureExtractArray = []
    for i, row in enumerate(dataSetArray):
        test = [row[2], row[8], row[9], row[10], row[11], row[12], row[13], row[14], row[15], row[16], row[17], row[18], row[19], row[20], row[4], row[21], row[22], row[23], row[24], row[25], row[26], row[27], row[28], row[29], row[30], row[31]]
        
        try:
        
            test = [float(ele) for ele in test]
        except:
            print(i, test)
        featureExtractArray.append(test)
    return featureExtractArray[1:].copy()
    #print(featureExtractArray)
    
#trainingDataSet feature extraction
featuretrainingDataSet = featureExtract(trainingDataSet)

#testDataSet feature extraction
featuretestDataSet = featureExtract(testDataSet)

#validationDataSet feature extraction
featurevalidationDataSet = featureExtract(validationDataSet)

In [7]:
print(featuretrainingDataSet[0])

[1272.0, 26.0, 62.0, 8.0, 20.0, 10.0, 19.0, 15.0, 28.0, 16.0, 13.0, 4.0, 4.0, 18.0, 1393.0, 24.0, 67.0, 6.0, 24.0, 9.0, 20.0, 20.0, 25.0, 7.0, 12.0, 8.0]


In [8]:
#randomization of winning and losing team statistics from feature trainingDataSet
#new label
randomizationLabelsTrainingSet = np.random.randint(2, size = len(featuretrainingDataSet))
#26 elements in featuretrainingDataSet

# randomization of winning and losing team statistics from feature testDataSet
randomizationLabelsTestSet = np.random.randint(2, size = len(featuretestDataSet))

#randomization of winning and losing team statistics from feature validationDataSet
randomizationLabelsValidationSet = np.random.randint(2, size = len(featurevalidationDataSet))

n_feat = len(featuretrainingDataSet[0])
assert n_feat % 2 == 0

#define as function
def randomization(labelList, originalDataSet):
    randomizedDataSet = []
    for i, do_flip in enumerate(labelList):    
        #if its a 1 that means the back half is lost
        x = originalDataSet[i] 
        if do_flip == 1:   
            x_rand = x[n_feat // 2:] + x[0:n_feat // 2]
        else:
            x_rand = x
        randomizedDataSet.append(x_rand)
    return randomizedDataSet

randomizedfeaturetrainingDataSet = randomization(randomizationLabelsTrainingSet,featuretrainingDataSet)
randomizedfeaturetestDataSet = randomization(randomizationLabelsTestSet, featuretestDataSet)
randomizedfeaturevalidationDataSet = randomization(randomizationLabelsValidationSet,featurevalidationDataSet)

#     for i, do_flip in enumerate(randomizationSeedTrainingSet):    
#         #if its a 1 that means the back half is wins
#         if do_flip == 1:
#             x = featuretrainingDataSet[i]        
#             x_rand = x[n_feat // 2:] + x[0:n_feat // 2]
#         else:
#             x_rand = x
#         randomizedfeaturetrainingDataSet.append(x_rand)

print(randomizedfeaturetrainingDataSet[0])
print(randomizedfeaturetestDataSet[0])
print(randomizedfeaturevalidationDataSet[0])

[1272.0, 26.0, 62.0, 8.0, 20.0, 10.0, 19.0, 15.0, 28.0, 16.0, 13.0, 4.0, 4.0, 18.0, 1393.0, 24.0, 67.0, 6.0, 24.0, 9.0, 20.0, 20.0, 25.0, 7.0, 12.0, 8.0]
[18.0, 1157.0, 23.0, 59.0, 6.0, 14.0, 11.0, 18.0, 15.0, 22.0, 6.0, 20.0, 8.0, 1103.0, 25.0, 59.0, 7.0, 22.0, 15.0, 26.0, 14.0, 24.0, 10.0, 17.0, 13.0, 5.0]
[1104.0, 29.0, 63.0, 7.0, 23.0, 17.0, 19.0, 14.0, 20.0, 17.0, 8.0, 14.0, 9.0, 16.0, 1406.0, 18.0, 50.0, 7.0, 21.0, 11.0, 15.0, 14.0, 20.0, 9.0, 22.0, 2.0]


In [10]:
print(featuretrainingDataSet[0])
print(randomizationLabelsTrainingSet[0])

[1272.0, 26.0, 62.0, 8.0, 20.0, 10.0, 19.0, 15.0, 28.0, 16.0, 13.0, 4.0, 4.0, 18.0, 1393.0, 24.0, 67.0, 6.0, 24.0, 9.0, 20.0, 20.0, 25.0, 7.0, 12.0, 8.0]
0


In [11]:
#pull labels out and then flip the respecitve lists within the data set corresponding to the data set
#save the array into np and then turn into torch
#implement logistical reggression

In [14]:
#converts to numpy array and pytorch aeeay
def npPytorchArray (randomizedDataSet, randomizedLabels):
    x_np = np.array(randomizedDataSet).astype(np.float32)
    y_np = np.array(randomizedLabels)
    x_pytorch = torch.from_numpy(x_np)
    y_pytorch = torch.from_numpy(y_np)    
    return x_pytorch, y_pytorch

randomizedfeaturetrainingDataSet_pytorch, randomizationLabelsTrainingSet_pytorch = npPytorchArray(randomizedfeaturetrainingDataSet, randomizationLabelsTrainingSet)
randomizedfeaturetestDataSet_pytorch, randomizationLabelsTestSet_pytorch = npPytorchArray(randomizedfeaturetestDataSet, randomizationLabelsTestSet)
randomizedfeaturevalidationDataSet_pytorch, randomizationLabelsValidationSet_pytorch = npPytorchArray(randomizedfeaturevalidationDataSet, randomizationLabelsValidationSet)

#save data set
torch.save(randomizedfeaturetrainingDataSet_pytorch, 'randomizedfeaturetraining_pytorch.pt')
torch.save(randomizedfeaturetestDataSet_pytorch, 'randomizedfeaturetest_pytorch.pt')
torch.save(randomizedfeaturevalidationDataSet_pytorch, 'randomizedfeaturevalidation_pytorch.pt')

#save labels
torch.save(randomizationLabelsTrainingSet_pytorch, 'randomizedlabelstraining_pytorch.pt')
torch.save(randomizationLabelsTestSet_pytorch, 'randomizedlabelstraining_pytorch.pt')
torch.save(randomizationLabelsValidationSet_pytorch, 'randomizedlabelstraining_pytorch.pt')

# print(y_np.shape)

# for i,x in enumerate(featuretrainingDataSet):
#     try:
#         np.array(x).astype(np.float32)
#     except:
#         print(i, x)


In [ ]:
def train(model, loss, optimizer, x_val, y_val):
    x = Variable(x_val, requires_grad=False)
    y = Variable(y_val, requires_grad=False)
    
train()